In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import sys

In [ ]:
class Histo :

    def __init__(self,xbin_edges,ybin_edges=[],zbin_edges=[]) :
        self.xbin_edges = xbin_edges
        shape = (len(self.xbin_edges)+1,)
        self.ybin_edges = []
        self.zbin_edges = []
        
        if len(ybin_edges) :
            self.ybin_edges = ybin_edges
            shape += ((len(self.ybin_edges)+1,))

            if len(zbin_edges) :
                self.zbin_edges = zbin_edges
                shape += ((len(self.zbin_edges)+1,))

        self.sumw = np.zeros(shape)
        self.sumw2 = np.zeros(shape)
        return
    
    @classmethod
    def FixedWidthConstructor(cls,nbinsx,xlow,xup,
                              nbinsy=0,ylow=None,yup=None,
                              nbinsz=0,zlow=None,zup=None) :
        """call via my_inst = Histo.FixedWidthConstructor(20,-1,1)"""
        
        xbin_edges = np.linspace(xlow, xup, num=nbinsx+1)
        ybin_edges = []
        zbin_edges = []
        
        if nbinsy :
            ybin_edges = np.linspace(ylow, yup, num=nbinsx+1)

            if nbinsz :
                zbin_edges = np.linspace(zlow, zup, num=nbinsz+1)
            
        return cls(xbin_edges,ybin_edges,zbin_edges)

    def Fill(self,*args) :
        xval = args[0]
        xbin = np.searchsorted(self.xbin_edges,xval,side='right')
        
        if not len(self.ybin_edges) :
            
            weight = 1 if len(args) <= 1 else args[1]
            self.sumw[xbin] += weight
            self.sumw2[xbin] += weight**2
            return

        yval = args[1]
        ybin = np.searchsorted(self.ybin_edges,yval,side='right')

        if not len(self.zbin_edges) :
            weight = 1 if len(args) <= 2 else args[2]
            self.sumw[xbin][ybin] += weight
            self.sumw2[xbin][ybin] += weight**2
            return

        zval = args[2]
        zbin = np.searchsorted(self.zbin_edges,zval,side='right')

        weight = 1 if len(args) <= 3 else args[3]
        self.sumw[xbin][ybin][zbin] += weight
        self.sumw2[xbin][ybin][zbin] += weight**2

        return

    def BinCenters(self,bin_edges) :
        return bin_edges[:-1] + np.diff(bin_edges) / 2

    def Draw(self) :
        import matplotlib.pyplot as plt
        plt.errorbar(self.BinCenters(self.xbin_edges), self.sumw[1:-1], yerr=np.sqrt(self.sumw2[1:-1]), fmt='o')


In [ ]:
ax = plt.subplot()
ax.set_yscale("log")

# First try: pyplot "hist". Eh... not good.
xx = np.random.normal(0,1,(10000))
plt.hist(xx,np.arange(-4,4,0.4))

# second try: np.histogram <- I like this one ... but it can be memory intensive
if True :
    x = np.random.randn(10000)
    print("size of x: {} kB".format(sys.getsizeof(x)/1000))
    counts,bin_edges = np.histogram(x,20,[-4,4])
    #bin_centers = (bin_edges[:-1] + bin_edges[1:])/2.
    bin_centers = bin_edges[:-1] + np.diff(bin_edges) / 2
    err = np.sqrt(counts)
    print("counts:",counts)
    print("errors:",err)
    plt.errorbar(bin_centers, counts, yerr=err, fmt='o')

# third try: keep track of sumw, sumw2 manually, and errorbar
# Histo class! (Supports 1D, 2D, 3D histograms)
if True :
    histo = Histo.FixedWidthConstructor(20,-4.0,4.0)
    print("size of histo: {} kB".format(sys.getsizeof(histo)))
    
    for i in range(10000) :
        val = np.random.randn()
        histo.Fill(val)
        
    histo.Draw()

plt.show()


In [ ]:
fig2,ax2 = plt.subplots()

# 2d histogram
if True :
    histo2 = Histo.FixedWidthConstructor(20,-4.0,4.0)
    
    for i in range(10000) :
        val = np.random.randn()
        histo2.Fill(val)
        
    plt.errorbar(histo2.BinCenters(histo2.xbin_edges),histo2.sumw[1:-1],
                 yerr=np.sqrt(histo2.sumw2[1:-1]), fmt='o',fillstyle='none')


plt.show()

In [ ]:
fig2,ax2 = plt.subplots()
ax2.set(xlabel='x axis', ylabel='y axis',title='Plot title')

# 2d histogram
#histo2d = Histo.FixedWidthConstructor(10,-4,4,10,-4,4)
histo2d = Histo(list(np.linspace(-4, 0, num=20))+list(np.linspace(0.4, 4, num=9)),
                list(np.linspace(-4, 0, num=20))+list(np.linspace(0.4, 4, num=9)))
#histo2d = Histo(list(np.linspace(-4, 0, num=2))+list(np.linspace(1, 4, num=3)),
#                list(np.linspace(-4, 0, num=2))+list(np.linspace(1, 4, num=3)))

for i in range(100000) :
    val1 = np.random.normal(-1)
    val2 = np.random.normal(-1)
    histo2d.Fill(val1,val2)

X,Y = np.meshgrid(histo2d.xbin_edges, histo2d.ybin_edges)
area = (X[:-1,1:]-X[:-1,:-1]) * (Y[1:,:-1]-Y[:-1,:-1])
sumw_width = histo2d.sumw[1:-1,1:-1] / area
plt.pcolor(X, Y, sumw_width,cmap='Blues')
cbar = plt.colorbar()
cbar.set_label('z axis', rotation=270,labelpad=15)
    
plt.show()

In [ ]:
fig2p1,ax2p1 = plt.subplots()
ax2p1.set(xlabel='x axis', ylabel='y axis',title='Plot title')

X,Y = np.meshgrid(histo2d.xbin_edges, histo2d.ybin_edges)
plt.pcolor(X, Y, histo2d.sumw[1:-1,1:-1],cmap='Blues')
plt.colorbar().set_label('z axis', rotation=270,labelpad=15)

plt.show()

In [ ]:
fig3,ax3 = plt.subplots()
ax3.set(xlabel='x axis', ylabel='y axis',title='Plot title')

X,Y = np.meshgrid(histo2d.BinCenters(histo2d.xbin_edges),
                  histo2d.BinCenters(histo2d.ybin_edges))

Xedges,Yedges = np.meshgrid(histo2d.xbin_edges, histo2d.ybin_edges)
area = (Xedges[:-1,1:]-Xedges[:-1,:-1]) * (Yedges[1:,:-1]-Yedges[:-1,:-1])

sumw_width = histo2d.sumw[1:-1,1:-1] / area

plt.contourf(X,Y,sumw_width,cmap='Blues')
ax3.set_xlim([-4,4])
ax3.set_ylim([-4,4])
plt.colorbar().set_label('z axis', rotation=270,labelpad=15)
plt.show()